In [ ]:
import os
import numpy as np
import rasterio
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import geopandas as gpd

# 1. Téléchargement des Images Sentinel-2
def download_sentinel_data(username, password, footprint, start_date, end_date):
    api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
    products = api.query(footprint,
                         date=(start_date, end_date),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    api.download_all(products)

# 2. Prétraitement des Images
def preprocess_image(image_path):
    with rasterio.open(image_path) as src:
        band_red = src.read(3)
        band_green = src.read(2)
        band_blue = src.read(1)
        band_nir = src.read(8)

        # Normalisation
        band_red = band_red / 10000.0
        band_green = band_green / 10000.0
        band_blue = band_blue / 10000.0
        band_nir = band_nir / 10000.0
        
        return np.stack([band_red, band_green, band_blue, band_nir], axis=-1)

# 3. Classification
def classify_image(image):
    # Placeholder for training data and labels
    X = []  # Features
    y = []  # Labels
    
    # Add your training data collection logic here

    # Example: Train a RandomForestClassifier
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)
    
    prediction = classifier.predict(image.reshape(-1, image.shape[2])).reshape(image.shape[0], image.shape[1])
    return prediction

# 4. Visualisation
def plot_classification(prediction, image_path):
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    with rasterio.open(image_path) as src:
        ax.imshow(src.read([3, 2, 1]).transpose(1, 2, 0) / 10000.0, extent=src.bounds, cmap='gray')
        ax.imshow(prediction, alpha=0.5, cmap='viridis')
    plt.show()

# Exemple d'utilisation
username = 'votre_nom_utilisateur'
password = 'votre_mot_de_passe'
footprint = geojson_to_wkt(read_geojson('votre_fichier_geojson.geojson'))
start_date = '20220101'
end_date = '20220131'

# Télécharger les données
download_sentinel_data(username, password, footprint, start_date, end_date)

# Prétraiter une image
image_path = 'chemin/vers/votre/image.tif'
image = preprocess_image(image_path)

# Classifier l'image
prediction = classify_image(image)

# Visualiser le résultat
plot_classification(prediction, image_path)
